In [8]:
import numpy as np
import pandas as pd
import time

d = pd.read_csv("temp.csv")
d.head()

,CD304,CD1c,FcER1,HLADR,Clec9a,CD14,56,CD15,CD203c,CD141,CD16
0,0.728950,1.800504,0.461779,0.105685,0.583352,0.021290,2.136260,1.112822,0.651236,0.504272,0.803037
1,1.055230,1.770434,0.595516,1.918719,1.186435,1.687472,2.020411,1.085909,0.691856,-0.218060,0.557938
2,0.608616,1.583415,0.864213,1.832889,0.944195,1.788921,1.844008,1.051284,0.809935,0.067364,1.926569
3,0.718539,1.892303,0.739363,1.828675,1.116597,1.756571,2.187723,1.013532,0.649286,0.810834,-0.143356
4,0.523798,1.865857,0.976382,1.941438,1.483872,1.736905,2.605964,1.835294,-0.486019,1.165125,1.901858


In [9]:
import scanpy as sc
import anndata
import igraph
d = d.iloc[:,:11]
adata = anndata.AnnData(X=pd.DataFrame.to_numpy(d), obs=list(d.index.values), var=list(d.columns.values))
adata

AnnData object with n_obs × n_vars = 284398 × 11 
    obs: 0
    var: 0

In [15]:
# print("Nearest neighbours...")
# start = time.time()
# sc.pp.neighbors(adata, n_neighbors=45)
# print("DONE: {} s".format(time.time()-start))
print("Louvain clustering...")
mid = time.time()
sc.tl.louvain(adata, resolution=0.13)
print("DONE: {} s".format(time.time()-mid))
print("Number of clusters: ",max(np.array(adata.obs['louvain']).astype(int)+1))
d['cluster']=np.array(adata.obs['louvain']).astype(int)+1
d.to_csv("temp.csv", index=False)

Louvain clustering...
DONE: 547.298743724823 s
Number of clusters:  5
